In [24]:
from headers import HEADERS
from green_app_lib import get_random_form, get_result, parse_content
from forms import FORM
import requests
import time
from multiprocessing import Pool, cpu_count
import random

def get_random_car():
    random_form = get_random_form(FORM)
    random_form["TransportViewModel.CarUsageList[0].SelectedFuelTypeOptionId"] = 15 # random.choice([15, 16, 17, 18])
    random_form["TransportViewModel.CarUsageList[0].AnnualMileage"] = random.randint(1, 250) * 200
    random_form["TransportViewModel.CarUsageList[0].AverageConsumption"] = random.randint(1, 11)
    return random_form

def get_record(form_field_step):
    form, field, step = form_field_step

    form_x = form.copy()
    form_x[field] = form[field] + step
    
    page1 = None
    page2 = None
    try:
        page1 = get_result(form)
        page2 = get_result(form_x)
    except Exception as e:
        print(e)
    
    result1 = None
    result2 = None
    if page1 != None:
        if page1.ok:
            result1 = parse_content(page1)
    if page2 != None:
        if page2.ok:
            result2 = parse_content(page2)
        
    if result1 != None and result2 != None:
        return (form['HouseholdViewModel.SelectedResidenceCountryId'],
                form["HouseholdViewModel.HouseholdSize"],
                'type_fuel = {}'.format(form['TransportViewModel.CarUsageList[0].SelectedFuelTypeOptionId']),
                'mileages = {}'.format(form['TransportViewModel.CarUsageList[0].AnnualMileage'] / 1000),
                'liters = {}'.format(form['TransportViewModel.CarUsageList[0].AverageConsumption']),
                'total_annual_dx = {}'.format((float(result2['total_annual']) - float(result1['total_annual'])) / form['TransportViewModel.CarUsageList[0].AverageConsumption']))
    return 0

# TransportViewModel.CarUsageList[0].AnnualMileage

In [25]:
field = "TransportViewModel.CarUsageList[0].AnnualMileage"
step = 1000
forms_field_step = []
for i in range(50000, 100000, 10000):
    form = get_random_car()
    form[field] = i
    forms_field_step.append((form, field, step))

rss = []
with Pool(32) as pool:
    rss = pool.map(get_record, forms_field_step)
    pool.close()
    pool.join()

for record in rss:
    print(record)

(33, 7, 'type_fuel = 15', 'mileages = 50.0', 'liters = 2', 'total_annual_dx = 0.024999999999991473')
(35, 1, 'type_fuel = 15', 'mileages = 60.0', 'liters = 8', 'total_annual_dx = 0.023750000000001492')
(35, 4, 'type_fuel = 15', 'mileages = 70.0', 'liters = 4', 'total_annual_dx = 0.02499999999999858')
(33, 2, 'type_fuel = 15', 'mileages = 80.0', 'liters = 2', 'total_annual_dx = 0.02499999999999858')
(33, 2, 'type_fuel = 15', 'mileages = 90.0', 'liters = 1', 'total_annual_dx = 0.01999999999999602')


# TransportViewModel.CarUsageList[0].AverageConsumption

In [26]:
field = "TransportViewModel.CarUsageList[0].AverageConsumption"
step = 1
forms_field_step = []
for i in range(1, 30000, 1000):
    form = get_random_car()
    form[field] = i
    forms_field_step.append((form, field, step))

rss = []
with Pool(32) as pool:
    rss = pool.map(get_record, forms_field_step)
    pool.close()
    pool.join()

for record in rss:
    print(record)

(80, 4, 'type_fuel = 15', 'mileages = 34.4', 'liters = 1', 'total_annual_dx = 2.3100000000000023')
(72, 7, 'type_fuel = 15', 'mileages = 39.2', 'liters = 1001', 'total_annual_dx = 0.0009290709290709927')
(45, 3, 'type_fuel = 15', 'mileages = 36.6', 'liters = 2001', 'total_annual_dx = 0.00043478260869559765')
(80, 5, 'type_fuel = 15', 'mileages = 21.4', 'liters = 3001', 'total_annual_dx = 0.00047650783072288624')
(33, 4, 'type_fuel = 15', 'mileages = 20.0', 'liters = 4001', 'total_annual_dx = 0.00011997000749813001')
(35, 6, 'type_fuel = 15', 'mileages = 3.2', 'liters = 5001', 'total_annual_dx = 1.3997200559875292e-05')
(35, 8, 'type_fuel = 15', 'mileages = 36.8', 'liters = 6001', 'total_annual_dx = 0.00014664222629563557')
(35, 2, 'type_fuel = 15', 'mileages = 30.4', 'liters = 7001', 'total_annual_dx = 0.00010284245107832384')
(45, 7, 'type_fuel = 15', 'mileages = 19.0', 'liters = 8001', 'total_annual_dx = 5.62429696288305e-05')
(72, 4, 'type_fuel = 15', 'mileages = 25.6', 'liters = 90